<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Stream Ensemble Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [1]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Metrics,Accuracy,BalancedAccuracy,CohenKappa,GeometricMean
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

## Online Bagging

---
[Online Bagging](https://riverml.xyz/latest/api/ensemble/BaggingClassifier/) is an Online bootstrap aggregation for classification.

In [4]:
from river.ensemble import BaggingClassifier
from hoeffding_tree_classifier import HoeffdingTreeClassifier

model = BaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 70.77%, BalancedAccuracy: 67.47%, GeometricMean: 0.668005, CohenKappa: 0.344538
[2,000] Accuracy: 70.29%, BalancedAccuracy: 66.94%, GeometricMean: 0.664196, CohenKappa: 0.323684
[3,000] Accuracy: 71.29%, BalancedAccuracy: 62.56%, GeometricMean: 0.590404, CohenKappa: 0.264543
[4,000] Accuracy: 71.82%, BalancedAccuracy: 61.73%, GeometricMean: 0.56612, CohenKappa: 0.257237
[5,000] Accuracy: 72.67%, BalancedAccuracy: 62.70%, GeometricMean: 0.576656, CohenKappa: 0.279668
[6,000] Accuracy: 73.18%, BalancedAccuracy: 63.95%, GeometricMean: 0.597133, CohenKappa: 0.303393
[7,000] Accuracy: 74.10%, BalancedAccuracy: 65.01%, GeometricMean: 0.610552, CohenKappa: 0.325977
[8,000] Accuracy: 73.83%, BalancedAccuracy: 64.93%, GeometricMean: 0.610232, CohenKappa: 0.323882
[9,000] Accuracy: 73.95%, BalancedAccuracy: 64.68%, GeometricMean: 0.603003, CohenKappa: 0.321963
[10,000] Accuracy: 74.11%, BalancedAccuracy: 64.59%, GeometricMean: 0.600334, CohenKappa: 0.321409
[11,000] Accuracy: 7

Accuracy: 74.68%, BalancedAccuracy: 66.38%, GeometricMean: 0.625314, CohenKappa: 0.357824

## Leveraging Bagging
---

[Leveraging Bagging](https://riverml.xyz/latest/api/ensemble/LeveragingBaggingClassifier/) is an improvement over the Oza Bagging algorithm. The bagging performance is leveraged by increasing the re-sampling. It uses a poisson distribution to simulate the re-sampling process. To increase re-sampling it uses a higher w value of the Poisson distribution (agerage number of events), 6 by default, increasing the input space diversity, by attributing a different range of weights to the data samples.

To deal with concept drift, Leveraging Bagging uses the ADWIN algorithm to monitor the performance of each member of the enemble If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [5]:
from river.ensemble import LeveragingBaggingClassifier
from hoeffding_tree_classifier import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 76.48%, BalancedAccuracy: 73.05%, GeometricMean: 0.723826, CohenKappa: 0.462855
[2,000] Accuracy: 78.04%, BalancedAccuracy: 71.86%, GeometricMean: 0.701805, CohenKappa: 0.455174
[3,000] Accuracy: 78.53%, BalancedAccuracy: 70.76%, GeometricMean: 0.683307, CohenKappa: 0.442877
[4,000] Accuracy: 78.02%, BalancedAccuracy: 69.92%, GeometricMean: 0.67075, CohenKappa: 0.429924
[5,000] Accuracy: 77.88%, BalancedAccuracy: 69.91%, GeometricMean: 0.670952, CohenKappa: 0.429222
[6,000] Accuracy: 77.81%, BalancedAccuracy: 70.48%, GeometricMean: 0.680915, CohenKappa: 0.435962
[7,000] Accuracy: 78.25%, BalancedAccuracy: 71.20%, GeometricMean: 0.690535, CohenKappa: 0.448589
[8,000] Accuracy: 77.85%, BalancedAccuracy: 70.79%, GeometricMean: 0.685591, CohenKappa: 0.440547
[9,000] Accuracy: 77.75%, BalancedAccuracy: 70.38%, GeometricMean: 0.678788, CohenKappa: 0.435653
[10,000] Accuracy: 77.98%, BalancedAccuracy: 70.44%, GeometricMean: 0.678725, CohenKappa: 0.438306
[11,000] Accuracy: 7

Accuracy: 78.24%, BalancedAccuracy: 72.22%, GeometricMean: 0.703804, CohenKappa: 0.467918

## AdaptiveRandomForest
---



The 3 most important aspects of [ARF](https://riverml.xyz/latest/api/ensemble/AdaptiveRandomForestClassifier/) are:
- inducing diversity through re-sampling
- inducing diversity through randomly selecting subsets of features for node splits
- drift detectors per base tree, which cause selective resets in response to drifts

It also allows training background trees, which start training if a warning is detected and replace the active tree if the warning escalates to a drift.

In [6]:
from adaptive_random_forest import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 73.57%, BalancedAccuracy: 65.76%, GeometricMean: 0.617855, CohenKappa: 0.343963
[2,000] Accuracy: 76.24%, BalancedAccuracy: 66.70%, GeometricMean: 0.623128, CohenKappa: 0.371088
[3,000] Accuracy: 77.33%, BalancedAccuracy: 66.64%, GeometricMean: 0.616279, CohenKappa: 0.376225
[4,000] Accuracy: 77.19%, BalancedAccuracy: 66.55%, GeometricMean: 0.61282, CohenKappa: 0.376537
[5,000] Accuracy: 77.20%, BalancedAccuracy: 66.79%, GeometricMean: 0.616567, CohenKappa: 0.380839
[6,000] Accuracy: 77.21%, BalancedAccuracy: 67.50%, GeometricMean: 0.630531, CohenKappa: 0.391378
[7,000] Accuracy: 77.97%, BalancedAccuracy: 68.60%, GeometricMean: 0.64618, CohenKappa: 0.413331
[8,000] Accuracy: 77.92%, BalancedAccuracy: 68.88%, GeometricMean: 0.650834, CohenKappa: 0.417829
[9,000] Accuracy: 77.75%, BalancedAccuracy: 68.59%, GeometricMean: 0.645828, CohenKappa: 0.413217
[10,000] Accuracy: 77.75%, BalancedAccuracy: 68.45%, GeometricMean: 0.643616, CohenKappa: 0.410647
[11,000] Accuracy: 77

Accuracy: 77.97%, BalancedAccuracy: 70.45%, GeometricMean: 0.675037, CohenKappa: 0.444296

## StreamingRandomPatches
---
[SRP](https://riverml.xyz/latest/api/ensemble/SRPClassifier/) is an ensemble method that simulates bagging or random subspaces. The default algorithm uses both bagging and random subspaces, namely Random Patches. The default base estimator is a Hoeffding Tree, but other base estimators can be used (differently from random forest variations).

In [7]:
from river.ensemble import SRPClassifier
from hoeffding_tree_classifier import HoeffdingTreeClassifier

model = SRPClassifier(model=HoeffdingTreeClassifier(),
                      n_models=10,
                      seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 74.77%, BalancedAccuracy: 67.28%, GeometricMean: 0.637276, CohenKappa: 0.375938
[2,000] Accuracy: 76.49%, BalancedAccuracy: 66.64%, GeometricMean: 0.619467, CohenKappa: 0.372717
[3,000] Accuracy: 77.89%, BalancedAccuracy: 67.07%, GeometricMean: 0.619651, CohenKappa: 0.388269
[4,000] Accuracy: 77.99%, BalancedAccuracy: 67.64%, GeometricMean: 0.627428, CohenKappa: 0.400094
[5,000] Accuracy: 78.06%, BalancedAccuracy: 67.94%, GeometricMean: 0.63199, CohenKappa: 0.405894
[6,000] Accuracy: 77.93%, BalancedAccuracy: 68.11%, GeometricMean: 0.635992, CohenKappa: 0.407224
[7,000] Accuracy: 78.37%, BalancedAccuracy: 68.79%, GeometricMean: 0.646277, CohenKappa: 0.420215
[8,000] Accuracy: 77.96%, BalancedAccuracy: 68.57%, GeometricMean: 0.644514, CohenKappa: 0.41425
[9,000] Accuracy: 77.91%, BalancedAccuracy: 68.44%, GeometricMean: 0.641418, CohenKappa: 0.412953
[10,000] Accuracy: 78.10%, BalancedAccuracy: 68.57%, GeometricMean: 0.642915, CohenKappa: 0.41615
[11,000] Accuracy: 78.

Accuracy: 78.35%, BalancedAccuracy: 70.51%, GeometricMean: 0.672847, CohenKappa: 0.44929

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [9]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## ADWIN Online Bagging

---
[ADWIN Online Bagging](https://riverml.xyz/latest/api/ensemble/ADWINBaggingClassifier/) is the online bagging method with the addition of the ADWIN algorithm as a change detector. If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [11]:
from river.ensemble import ADWINBaggingClassifier
from hoeffding_tree_classifier import HoeffdingTreeClassifier

model = ADWINBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 81.58%, BalancedAccuracy: 74.02%, GeometricMean: 0.704663, CohenKappa: 0.536913
[2,000] Accuracy: 86.14%, BalancedAccuracy: 82.00%, GeometricMean: 0.809692, CohenKappa: 0.674807
[3,000] Accuracy: 87.23%, BalancedAccuracy: 83.85%, GeometricMean: 0.832175, CohenKappa: 0.703022
[4,000] Accuracy: 88.45%, BalancedAccuracy: 85.52%, GeometricMean: 0.850896, CohenKappa: 0.731245
[5,000] Accuracy: 88.52%, BalancedAccuracy: 85.62%, GeometricMean: 0.851896, CohenKappa: 0.733577
[6,000] Accuracy: 83.41%, BalancedAccuracy: 80.88%, GeometricMean: 0.802302, CohenKappa: 0.636114
[7,000] Accuracy: 80.48%, BalancedAccuracy: 78.94%, GeometricMean: 0.784531, CohenKappa: 0.589459
[8,000] Accuracy: 79.11%, BalancedAccuracy: 78.21%, GeometricMean: 0.778868, CohenKappa: 0.570452
[9,000] Accuracy: 77.32%, BalancedAccuracy: 76.88%, GeometricMean: 0.767027, CohenKappa: 0.54051
[10,000] Accuracy: 75.84%, BalancedAccuracy: 75.62%, GeometricMean: 0.755134, CohenKappa: 0.513835
[11,000] Accuracy: 7

Accuracy: 73.14%, BalancedAccuracy: 72.81%, GeometricMean: 0.725466, CohenKappa: 0.458589

## Leveraging Bagging

In [12]:
from river.ensemble import LeveragingBaggingClassifier
from hoeffding_tree_classifier import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 73.17%, BalancedAccuracy: 65.54%, GeometricMean: 0.614105, CohenKappa: 0.339901
[2,000] Accuracy: 77.89%, BalancedAccuracy: 71.38%, GeometricMean: 0.683953, CohenKappa: 0.464885
[3,000] Accuracy: 79.96%, BalancedAccuracy: 73.86%, GeometricMean: 0.714632, CohenKappa: 0.515299
[4,000] Accuracy: 81.95%, BalancedAccuracy: 76.14%, GeometricMean: 0.741993, CohenKappa: 0.561654
[5,000] Accuracy: 83.22%, BalancedAccuracy: 78.06%, GeometricMean: 0.765253, CohenKappa: 0.597349
[6,000] Accuracy: 78.45%, BalancedAccuracy: 74.95%, GeometricMean: 0.736033, CohenKappa: 0.520805
[7,000] Accuracy: 75.97%, BalancedAccuracy: 74.10%, GeometricMean: 0.733508, CohenKappa: 0.492701
[8,000] Accuracy: 74.92%, BalancedAccuracy: 73.91%, GeometricMean: 0.734936, CohenKappa: 0.483979
[9,000] Accuracy: 74.24%, BalancedAccuracy: 73.75%, GeometricMean: 0.735227, CohenKappa: 0.477857
[10,000] Accuracy: 73.60%, BalancedAccuracy: 73.38%, GeometricMean: 0.732717, CohenKappa: 0.468843
[11,000] Accuracy: 

Accuracy: 73.68%, BalancedAccuracy: 73.62%, GeometricMean: 0.736103, CohenKappa: 0.47227

## AdaptiveRandomForest

In [13]:
from adaptive_random_forest import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10,nominal_attributes=['elevel', 'car', 'zipcode'])
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 72.47%, BalancedAccuracy: 64.71%, GeometricMean: 0.60385, CohenKappa: 0.322104
[2,000] Accuracy: 75.84%, BalancedAccuracy: 68.87%, GeometricMean: 0.652994, CohenKappa: 0.4123
[3,000] Accuracy: 76.86%, BalancedAccuracy: 69.59%, GeometricMean: 0.659494, CohenKappa: 0.43053
[4,000] Accuracy: 78.74%, BalancedAccuracy: 71.66%, GeometricMean: 0.685575, CohenKappa: 0.474192
[5,000] Accuracy: 79.38%, BalancedAccuracy: 72.52%, GeometricMean: 0.695778, CohenKappa: 0.492122
[6,000] Accuracy: 74.88%, BalancedAccuracy: 69.68%, GeometricMean: 0.664408, CohenKappa: 0.423866
[7,000] Accuracy: 73.14%, BalancedAccuracy: 70.48%, GeometricMean: 0.688771, CohenKappa: 0.424788
[8,000] Accuracy: 72.23%, BalancedAccuracy: 70.95%, GeometricMean: 0.702484, CohenKappa: 0.426043
[9,000] Accuracy: 71.67%, BalancedAccuracy: 71.14%, GeometricMean: 0.708659, CohenKappa: 0.425568
[10,000] Accuracy: 70.97%, BalancedAccuracy: 70.79%, GeometricMean: 0.707188, CohenKappa: 0.416524
[11,000] Accuracy: 68.7

Accuracy: 69.35%, BalancedAccuracy: 69.26%, GeometricMean: 0.692444, CohenKappa: 0.385285

## StreamingRandomPatches
---
We set the drift and warning detection options

In [14]:
from river.ensemble import SRPClassifier
from hoeffding_tree_classifier import HoeffdingTreeClassifier
from river.drift import ADWIN

model = SRPClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                      n_models=10,
                      drift_detector=ADWIN(delta=0.001),
                      warning_detector=ADWIN(delta=0.01),
                      seed=42)

metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 89.29%, BalancedAccuracy: 85.81%, GeometricMean: 0.851739, CohenKappa: 0.748235
[2,000] Accuracy: 91.75%, BalancedAccuracy: 89.38%, GeometricMean: 0.890658, CohenKappa: 0.810822
[3,000] Accuracy: 92.43%, BalancedAccuracy: 90.39%, GeometricMean: 0.901719, CohenKappa: 0.826536
[4,000] Accuracy: 93.02%, BalancedAccuracy: 91.20%, GeometricMean: 0.910467, CohenKappa: 0.839381
[5,000] Accuracy: 93.28%, BalancedAccuracy: 91.59%, GeometricMean: 0.914569, CohenKappa: 0.845964
[6,000] Accuracy: 87.50%, BalancedAccuracy: 86.57%, GeometricMean: 0.864886, CohenKappa: 0.733074
[7,000] Accuracy: 84.43%, BalancedAccuracy: 84.17%, GeometricMean: 0.841526, CohenKappa: 0.679959
[8,000] Accuracy: 82.55%, BalancedAccuracy: 82.56%, GeometricMean: 0.825626, CohenKappa: 0.647289
[9,000] Accuracy: 80.85%, BalancedAccuracy: 80.98%, GeometricMean: 0.809676, CohenKappa: 0.616305
[10,000] Accuracy: 79.18%, BalancedAccuracy: 79.36%, GeometricMean: 0.792921, CohenKappa: 0.584297
[11,000] Accuracy: 

Accuracy: 80.50%, BalancedAccuracy: 80.58%, GeometricMean: 0.805648, CohenKappa: 0.609963

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.